In [1]:
import requests
import os
import json
import random 
import time
from datetime import datetime
import psycopg2
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_1641/3204136505.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Database connection parameters
dbname = "colruyt_tracker"
user = "thibodebras"
password = "postgres"
host = "localhost"  # or the IP address of your PostgreSQL server
port = "5432"

In [3]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

query = "SELECT * FROM raw_data_2024_06_23"

# Use read_sql_query to execute the query and assign the result to a DataFrame
df_raw = pd.read_sql_query(query, conn)

# Don't forget to close the connection when done
conn.close()

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_1641/4028714431.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql_query(query, conn)


In [4]:
df_raw.head()

,run_date,shop_id,page,products
0,2024-06-23,459,1,"[{'productId': '1038', 'technicalArticleNumber..."
1,2024-06-23,459,2,"[{'productId': '11170', 'technicalArticleNumbe..."
2,2024-06-23,459,3,"[{'productId': '11788', 'technicalArticleNumbe..."
3,2024-06-23,459,4,"[{'productId': '50461', 'technicalArticleNumbe..."
4,2024-06-23,459,5,"[{'productId': '535920', 'technicalArticleNumb..."


In [5]:
exploded_df = df_raw.explode('products')

products_normalized = pd.json_normalize(exploded_df['products'].apply(lambda x: x if isinstance(x, dict) else {}))

# Drop the 'products' column from exploded_df since it's no longer needed in its original form
exploded_df.drop(columns=['products'], inplace=True)

# Reset the index of exploded_df to merge it with products_normalized
exploded_df.reset_index(drop=True, inplace=True)

# Concatenate the normalized products DataFrame with the exploded DataFrame
# It's important to reset the index on products_normalized if you haven't already, to ensure alignment
result_df = pd.concat([exploded_df, products_normalized.reset_index(drop=True)], axis=1)
result_df.head()

,run_date,shop_id,page,productId,technicalArticleNumber,commercialArticleNumber,name,description,thumbNail,fullImage,squareImage,walkRouteSequenceNumber,isAvailable,isPriceAvailable,inPromo,gtin,topCategoryName,topCategoryId,businessDomain,IsPrivateLabel,IsBiffe,WeightconversionFactor,IsWeightArticle,IsBio,CountryOfOrigin,IsExclusivelySoldInLuxembourg,OrderUnit,ShortName,IsNew,LongName,RecentQuanityOfStockUnits,FicCode,price.basicPrice,price.recommendedQuantity,price.quantityPrice,price.quantityPriceQuantity,price.measurementUnitPrice,price.measurementUnitQuantityPrice,price.measurementUnit,price.isRedPrice,price.pricePerUOM,price.activationDate,price.recordSource,price.isPromoActive,price.priceChangeCode,price.quantityPricePerUOM,price.quantityActivationDate,price.quantityPriceChangeCode,brand,seoBrand,content,nutriscoreLabel,AlcoholVolume,price.unit,InSeason,StartSeasonDate,EcoscoreLabel,EcoscoreValue,promotion,DepositValue,price.redPriceReason,price.referencePrice,price.retentionPeriod,EndSeasonDate,BiffeDate
0,2024-06-23,459,1,1038,29750,29750,concombre,"*Origine: Belgique, Pays-Bas\n*Classe 1",https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,9884.0,True,True,False,"[08437011296795, 08436540550071, 0540014109581...",Légumes et fruits,1675,RETAIL_BE,False,False,0,False,False,BELGIQUE,False,P,CONCOMBRES PIECE,False,concombre,1,E,0.79,1.0,0.595,2.0,0.79,0.6,S,False,0.79,18-06-2024,Offline,N,L,0.6,18-06-2024,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-23,459,1,2205,29015,29015,bananes,"*Origine: Équateur, Colombie,\nCosta Rica\n*Cl...",https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,9415.0,True,True,False,"[05400141059267, 05400141256529, 0540014137505...",Légumes et fruits,1675,RETAIL_BE,True,False,1000,True,False,COLOMBIE,False,P,BANANES EVERYDAY 1KG P,False,EVERYDAY bananes ±1kg,"1,1,1,1",I,1.29,1.0,NaN,NaN,1.29,NaN,K,False,1.29,15-05-2024,Offline,N,L,NaN,NaN,NaN,EVERYDAY,Everyday,±1kg,A,0,K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-23,459,1,2204,29013,29013,bananes,"*Origine: Guatemala, Équateur,\nCosta Rica, Co...",https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,9414.0,True,True,False,"[05400141532067, 05400141978933, 0540014121938...",Légumes et fruits,1675,RETAIL_BE,True,False,1000,True,False,GUATEMALA,False,P,BONI BANANEN W,False,BONI bananes ±1kg,"1,1,1,1,1",I,1.99,1.2,NaN,NaN,1.99,NaN,K,False,1.99,20-06-2024,Offline,N,E,NaN,NaN,NaN,BONI,Boni Selection,±1kg,A,0,K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-23,459,1,2639,29247,29247,fraises,"*Origine: Belgique,\nPays-Bas\n*Classe 1",https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,9763.0,True,True,False,"[05400357500003, 05400141050578, 0540014126974...",Légumes et fruits,1675,RETAIL_BE,False,False,0,False,False,BELGIQUE,False,P,FRAISES A 500GR,False,fraises 500g,"1,8,1,1,1,1",E,2.99,1.0,NaN,NaN,5.98,NaN,K,False,5.98,20-06-2024,Offline,N,E,NaN,NaN,NaN,NaN,NaN,500g,A,0,NaN,True,23/03/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-06-23,459,1,1027,29707,29707,courgettes,"*Origine: Espagne,\nBelgique, Pays-Bas\n*Classe 1",https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,10205.0,True,True,False,"[08437011296504, 08435320640308, 0843654055006...",Légumes et fruits,1675,RETAIL_BE,False,False,300,True,False,ESPAGNE,False,"P,Kg",COURGETTES P,False,courgettes,1,N,1.97,1.0,NaN,NaN,1.97,NaN,K,False,1.97,14-06-2024,Offline,N,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
mask = result_df['technicalArticleNumber'] != result_df['commercialArticleNumber']

# Filter out the rows using the mask
filtered_df = result_df[mask]
filtered_df.head()

,run_date,shop_id,page,productId,technicalArticleNumber,commercialArticleNumber,name,description,thumbNail,fullImage,squareImage,walkRouteSequenceNumber,isAvailable,isPriceAvailable,inPromo,gtin,topCategoryName,topCategoryId,businessDomain,IsPrivateLabel,IsBiffe,WeightconversionFactor,IsWeightArticle,IsBio,CountryOfOrigin,IsExclusivelySoldInLuxembourg,OrderUnit,ShortName,IsNew,LongName,RecentQuanityOfStockUnits,FicCode,price.basicPrice,price.recommendedQuantity,price.quantityPrice,price.quantityPriceQuantity,price.measurementUnitPrice,price.measurementUnitQuantityPrice,price.measurementUnit,price.isRedPrice,price.pricePerUOM,price.activationDate,price.recordSource,price.isPromoActive,price.priceChangeCode,price.quantityPricePerUOM,price.quantityActivationDate,price.quantityPriceChangeCode,brand,seoBrand,content,nutriscoreLabel,AlcoholVolume,price.unit,InSeason,StartSeasonDate,EcoscoreLabel,EcoscoreValue,promotion,DepositValue,price.redPriceReason,price.referencePrice,price.retentionPeriod,EndSeasonDate,BiffeDate
5,2024-06-23,459,1,843058,4688137,30635,élevées au sol M,NaN,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,8932.0,True,True,False,"[45400141530948, 75400141530949, 1540014153094...",Crémerie,65,RETAIL_BE,True,False,0,False,False,BELGIQUE,False,P,EVD SCHARRELEI M 12ST,False,EVERYDAY élevées au sol M 12pc,"1,15,15,15,15,15",I,2.39,1.0,NaN,NaN,0.200,NaN,S,False,0.200,06-06-2024,Offline,N,H,NaN,NaN,NaN,EVERYDAY,Everyday,12pc,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-06-23,459,1,781866,4578220,28941,BONBONS TOMATES,*Origine: Belgique\n*Classe 1,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,10187.0,True,True,False,"[05400141719994, 05400141625097, 0540014168509...",Légumes et fruits,1675,RETAIL_BE,True,False,0,False,False,BELGIQUE,False,P,BONI BONB.TOMAT.CER500G,False,BONI BONBONS TOMATES 500g,"1,1,1,1",N,1.59,1.0,NaN,NaN,3.180,NaN,K,False,3.180,22-06-2024,Offline,N,L,NaN,NaN,NaN,BONI,Boni Selection,500g,A,0,NaN,True,12/07/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2024-06-23,459,1,3989,1106651,29809,tomates en grappe,*Origine: Belgique\n*Classe 1,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,9856.0,True,True,False,"[98426756096053, 08435563308942, 9843653391651...",Légumes et fruits,1675,RETAIL_BE,False,False,100,True,False,BELGIQUE,False,Kg,TOMATES GRAPPE P,False,tomates en grappe,1,N,1.49,1.0,NaN,NaN,1.490,NaN,K,False,1.490,20-06-2024,Offline,N,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2024-06-23,459,1,113780,3010360,15840,Avocat Ready2Eat,"*Origine: Mexique, Colombie, Pérou,\nAfrique d...",https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,9564.0,True,True,False,"[05425016494233, 08718963282693, 8540027900130...",Légumes et fruits,1675,RETAIL_BE,False,False,0,False,False,COLOMBIE,False,P,HASS AVOCAT RTE,False,HASS Avocat Ready2Eat,"1,40",E,0.99,1.0,NaN,NaN,0.990,NaN,S,False,0.990,20-06-2024,NaN,Y,E,NaN,NaN,NaN,HASS,Hass,NaN,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2024-06-23,459,1,13305,1331384,5125,Eau plate,*Eau de source\n*Non pétillante,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,8874.0,True,True,False,"[15400141971672, 05400141026849, 0540014108014...",Boissons,354,RETAIL_BE,True,False,0,False,False,FRANCE,False,P,EVD EAU SOURCE PLAT 2L,False,EVERYDAY Eau plate 2L,"6,1,1,384,6",I,0.39,6.0,NaN,NaN,0.195,NaN,L,False,0.195,04-06-2024,Offline,N,E,NaN,NaN,NaN,EVERYDAY,Everyday,2L,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
result_df[['WeightconversionFactor','IsWeightArticle']].drop_duplicates()
result_df['OrderUnit'].unique()

array(['P', 'P,Kg', 'Kg', nan], dtype=object)

In [6]:
result_df = result_df.drop_duplicates(subset=["shop_id", "productId"])

In [50]:
54572-54364

208

In [51]:
result_df.describe()


,page,walkRouteSequenceNumber,price.basicPrice,price.quantityPrice,price.measurementUnitPrice,price.measurementUnitQuantityPrice,price.pricePerUOM,price.quantityPricePerUOM,price.referencePrice,price.retentionPeriod
count,55832.000000,46527.000000,46308.000000,6441.000000,46300.000000,6441.000000,46300.000000,6441.000000,262.000000,262.0
mean,28.427604,8287.223677,6.820567,4.998246,18.100343,10.089960,18.100343,10.089960,10.154580,30.0
std,16.124120,4884.062688,7.750719,4.754318,57.521922,11.992682,57.521922,11.992682,21.269821,0.0
min,1.000000,1.000000,0.000000,0.260000,0.002000,0.004000,0.002000,0.004000,1.690000,30.0
25%,14.000000,3892.000000,2.550000,2.100000,3.990000,3.310000,3.990000,3.310000,3.090000,30.0
50%,28.000000,8607.000000,4.190000,3.350000,9.320000,7.140000,9.320000,7.140000,5.990000,30.0
75%,42.000000,12250.000000,7.990000,6.290000,17.600000,12.850000,17.600000,12.850000,11.030000,30.0
max,56.000000,18576.000000,120.000000,36.760000,2055.000000,279.000000,2055.000000,279.000000,199.000000,30.0


In [52]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55832 entries, 0 to 96027
Data columns (total 65 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   run_date                            55832 non-null  object 
 1   shop_id                             55832 non-null  object 
 2   page                                55832 non-null  int64  
 3   productId                           55832 non-null  object 
 4   technicalArticleNumber              55832 non-null  object 
 5   commercialArticleNumber             55832 non-null  object 
 6   name                                55752 non-null  object 
 7   description                         37482 non-null  object 
 8   thumbNail                           55832 non-null  object 
 9   fullImage                           55832 non-null  object 
 10  squareImage                         55832 non-null  object 
 11  walkRouteSequenceNumber             46527 non-

In [53]:
nameGroupDf = result_df.groupby("productId").agg({'name': lambda x: list(x.unique())})
filtered_result = nameGroupDf[nameGroupDf['name'].apply(lambda x: len(x) > 1)]
filtered_result

,name
productId,


In [54]:
result_df[result_df['name'].isnull() | result_df['ShortName'].isnull()].head(84)
result_df[result_df['ShortName'].isnull()].head(84)

,run_date,shop_id,page,productId,technicalArticleNumber,commercialArticleNumber,name,description,thumbNail,fullImage,squareImage,walkRouteSequenceNumber,isAvailable,isPriceAvailable,inPromo,gtin,topCategoryName,topCategoryId,businessDomain,IsPrivateLabel,IsBiffe,WeightconversionFactor,IsWeightArticle,IsBio,CountryOfOrigin,IsExclusivelySoldInLuxembourg,OrderUnit,ShortName,IsNew,LongName,RecentQuanityOfStockUnits,FicCode,price.basicPrice,price.recommendedQuantity,price.quantityPrice,price.quantityPriceQuantity,price.measurementUnitPrice,price.measurementUnitQuantityPrice,price.measurementUnit,price.isRedPrice,price.pricePerUOM,price.activationDate,price.recordSource,price.isPromoActive,price.priceChangeCode,price.quantityPricePerUOM,price.quantityActivationDate,price.quantityPriceChangeCode,brand,seoBrand,content,nutriscoreLabel,AlcoholVolume,price.unit,InSeason,StartSeasonDate,EcoscoreLabel,EcoscoreValue,promotion,DepositValue,price.redPriceReason,price.referencePrice,price.retentionPeriod,EndSeasonDate,BiffeDate
2107,2024-06-23,459,9,828356,4667028,4590,pâtes fantasia,*Italie,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,11644.0,True,True,False,"[05400141452136, 15400141452133]",Epicerie,233,RETAIL_BE,True,False,0,False,False,ITALIE,False,P,NaN,False,NaN,1,I,1.59,1.0,NaN,NaN,3.18,NaN,K,False,3.18,18-06-2024,NaN,N,H,NaN,NaN,NaN,BONI,Boni Selection,500g,A,NaN,NaN,NaN,NaN,B,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2024-06-23,1825,9,828356,4667028,4590,pâtes fantasia,*Italie,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,5133.0,True,True,False,"[05400141452136, 15400141452133]",Epicerie,233,RETAIL_BE,True,False,0,False,False,ITALIE,False,P,NaN,False,NaN,1,I,1.59,1.0,NaN,NaN,3.18,NaN,K,False,3.18,18-06-2024,NaN,N,H,NaN,NaN,NaN,BONI,Boni Selection,500g,A,NaN,NaN,NaN,NaN,B,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30033,2024-06-23,471,9,828356,4667028,4590,pâtes fantasia,*Italie,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,4166.0,True,True,False,"[05400141452136, 15400141452133]",Epicerie,233,RETAIL_BE,True,False,0,False,False,ITALIE,False,P,NaN,False,NaN,1,I,1.59,1.0,NaN,NaN,3.18,NaN,K,False,3.18,18-06-2024,NaN,N,H,NaN,NaN,NaN,BONI,Boni Selection,500g,A,NaN,NaN,NaN,NaN,B,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43996,2024-06-23,1671,9,828356,4667028,4590,pâtes fantasia,*Italie,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,4139.0,True,True,False,"[05400141452136, 15400141452133]",Epicerie,233,RETAIL_BE,True,False,0,False,False,ITALIE,False,P,NaN,False,NaN,1,I,1.59,1.0,NaN,NaN,3.18,NaN,K,False,3.18,18-06-2024,NaN,N,H,NaN,NaN,NaN,BONI,Boni Selection,500g,A,NaN,NaN,NaN,NaN,B,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# result_df.loc[result_df['shop_id'] == '459']

In [56]:
# result_df.groupby(result_df["productId"]).count().sort_values("run_date")

In [57]:
# result_df.loc[result_df["productId"]=="192790"]

In [58]:
# grouped = result_df.groupby(["productId"]).count()
# grouped.loc[grouped["run_date"]>1]

In [59]:
# result = result_df.groupby('productId')['price.quantityActivationDate'].nunique()
# result.sort_values()

In [60]:
# result_df.loc[result_df['productId'] == '10']

In [61]:
# result_df.columns.to_list

In [62]:
# OUT DATED 

products_df = result_df[['productId', 'name', 'topCategoryName', 'brand']].drop_duplicates()
products_df.columns = ['ProductID', 'ProductName', 'Category', 'Brand']

# Creating the Prices DataFrame
prices_df = result_df[['productId', 'shop_id', 'price.basicPrice', 'run_date']]
prices_df.columns = ['ProductID', 'ShopID', 'Price', 'Date']

# Creating a simple Shops DataFrame
shops_df = pd.DataFrame(result_df['shop_id'].unique(), columns=['ShopID'])
# Placeholder columns for Shop details (to be updated/filled elsewhere)
shops_df['ShopName'] = np.nan
shops_df['Address'] = np.nan
shops_df['City'] = np.nan

# Resetting indexes
products_df.reset_index(drop=True, inplace=True)
prices_df.reset_index(drop=True, inplace=True)
shops_df.reset_index(drop=True, inplace=True)

# products_df = products_df.set_index(["ProductID"], drop=True)
prices_df['Combined'] = prices_df['ProductID'].astype(str) + '-' + prices_df['ShopID'].astype(str) + '-' + prices_df['Date'].astype(str)


# Step 2: Hash the combined column
# We're using the built-in hash function for simplicity. For consistent results across sessions,
# consider using a stable hash function like hashlib or any other stable hashing mechanism.
prices_df['PriceID'] = prices_df['Combined'].apply(lambda x: hash(x))

# Step 3: Set the hash column as the new index
# prices_df = prices_df.set_index('PriceID')

# Dropping the Combined column as it's no longer needed, unless you want to keep it for reference
prices_df.drop('Combined', axis=1, inplace=True)
# shops_df = shops_df.set_index(["ShopID"], drop=True)

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_82362/2590782205.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['Combined'] = prices_df['ProductID'].astype(str) + '-' + prices_df['ShopID'].astype(str) + '-' + prices_df['Date'].astype(str)
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_82362/2590782205.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['PriceID'] = prices_df['Combined'].apply(lambda x: hash(x))
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/

In [63]:
products_df.head()

,ProductID,ProductName,Category,Brand
0,1038,concombre,Légumes et fruits,NaN
1,2205,bananes,Légumes et fruits,EVERYDAY
2,2204,bananes,Légumes et fruits,BONI
3,2639,fraises,Légumes et fruits,NaN
4,1027,courgettes,Légumes et fruits,NaN


In [64]:
prices_df.head()

,ProductID,ShopID,Price,Date,PriceID
0,1038,459,0.79,2024-06-23,5527235774738387675
1,2205,459,1.29,2024-06-23,1949494219402720820
2,2204,459,1.99,2024-06-23,-8635023651859170240
3,2639,459,2.99,2024-06-23,-197553616573996590
4,1027,459,1.97,2024-06-23,-7439397775601838816


In [65]:
shops_df

,ShopID,ShopName,Address,City
0,459,NaN,NaN,NaN
1,1825,NaN,NaN,NaN
2,471,NaN,NaN,NaN
3,1671,NaN,NaN,NaN


In [247]:
# from sqlalchemy import create_engine

# # PostgreSQL connection string format
# connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"
# engine = create_engine(connection_string)

In [248]:
# Write Products DataFrame to PostgreSQL
# products_df.to_sql('product_test', con=engine, if_exists='replace', index=True)

# # Write Prices DataFrame to PostgreSQL
# prices_df.to_sql('price_test', con=engine, if_exists='replace', index=True)

# # Write Shops DataFrame to PostgreSQL
# shops_df.to_sql('shop_test', con=engine, if_exists='replace', index=True)

In [249]:
# engine.dispose()

In [250]:
def convert_row_to_tuple(row):
    return tuple(row.astype(object)
                  .where(pd.notnull(row), None)
                  .apply(lambda x: x.item() if isinstance(x, np.generic) else x))

# Adjusted function for writing a DataFrame to PostgreSQL, including type conversion
def write_dataframe_to_postgresql(df, table_name, conn):
    # Convert DataFrame columns to native Python types to avoid numpy.int64 issue
    df = df.map(lambda x: x.item() if isinstance(x, np.generic) else x)
    
    # Prepare data for insertion
    records = [convert_row_to_tuple(row) for index, row in df.iterrows()]
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"
    
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, records)
        conn.commit()
    except Exception as e:
        print(f"Error inserting into {table_name}: {e}")
        conn.rollback()
    finally:
        cursor.close()



In [223]:

# def write_dataframe_to_postgresql(df, table_name, conn):
#     # Convert the dataframe to a list of tuples
#     records = list(df.to_records(index=True))

#     # Create a list of placeholders for each record
#     placeholders = ', '.join(['%s'] * len(df.columns))

#     # Create the INSERT INTO statement
#     columns = ', '.join(df.columns)
#     sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

#     # Create a cursor and execute the SQL statement
#     cursor = conn.cursor()
#     try:
#         cursor.executemany(sql, records)
#         conn.commit()  # Commit the transaction
#     except Exception as e:
#         conn.rollback()  # Rollback the transaction on error
#         print(f"Error inserting into {table_name}: {e}")
#     finally:
#         cursor.close()


In [252]:

# Example usage
write_dataframe_to_postgresql(products_df, 'product_test_2', conn)
print("1")

write_dataframe_to_postgresql(shops_df, 'shop_test_2', conn)
print("2")
write_dataframe_to_postgresql(prices_df, 'price_test_2', conn)
print("3")

1
2
3


In [66]:
def create_tables(conn):
    # SQL statements for creating the tables
    commands = (
        """
        CREATE TABLE IF NOT EXISTS Product (
            ProductID INT PRIMARY KEY,
            Name VARCHAR(255),
            LongName VARCHAR(255),
            ShortName VARCHAR(100),
            Description TEXT,
            TopCategoryName VARCHAR(255),
            TopCategoryId INT,
            Brand VARCHAR(255),
            SeoBrand VARCHAR(255)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Shop (
            ShopID INT PRIMARY KEY,
            ShopName VARCHAR(255) NOT NULL,
            Address VARCHAR(255),
            City VARCHAR(100)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS PriceHistory (
            PriceID VARCHAR(32) PRIMARY KEY,
            EffectiveDate DATE NOT NULL,
            ProductID INT NOT NULL,
            ShopID INT NOT NULL,
            BasicPrice DECIMAL(10, 2),
            RecommendedQuantity DECIMAL(10, 2),
            QuantityPrice DECIMAL(10, 2),
            QuantityPriceQuantity DECIMAL(10, 2),
            MeasurementUnitPrice DECIMAL(10, 2),
            MeasurementUnitQuantityPrice DECIMAL(10, 2),
            MeasurementUnit VARCHAR(50),
            IsRedPrice BOOLEAN,
            PricePerUOM DECIMAL(10, 2),
            ActivationDate VARCHAR(50),
            RecordSource VARCHAR(50),
            IsPromoActive VARCHAR(50),
            PriceChangeCode VARCHAR(10),
            QuantityPricePerUOM DECIMAL(10, 2),
            QuantityActivationDate VARCHAR(50),
            QuantityPriceChangeCode VARCHAR(10),
            RedPriceReason VARCHAR(255),
            ReferencePrice DECIMAL(10, 2),
            RetentionPeriod DECIMAL(10, 2),
            Unit VARCHAR(50),
            FOREIGN KEY (ProductID) REFERENCES Product(ProductID),
            FOREIGN KEY (ShopID) REFERENCES Shop(ShopID)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS ProductHistory (
            ProductHistoryID INT PRIMARY KEY,
            EffectiveDate DATE NOT NULL,
            ProductID INT NOT NULL,
            IsAvailable BOOLEAN,
            IsPriceAvailable BOOLEAN,
            InPromo BOOLEAN,
            IsBio BOOLEAN,
            CountryOfOrigin VARCHAR(100),
            IsNew BOOLEAN,
            Content TEXT,
            NutriscoreLabel VARCHAR(10),
            EcoscoreLabel VARCHAR(10),
            EcoscoreValue DECIMAL(5, 2),
            FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
        );
        """
    )
    cur = conn.cursor()
    try:
        # Execute each command
        for command in commands:
            cur.execute(command)
        cur.close()
        conn.commit()  # Commit the changes to the database
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()  # Rollback on error
    finally:
        if cur is not None:
            cur.close()


In [138]:

conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)


In [139]:

# Call the function to create the tables
create_tables(conn)

In [ ]:
# CREATE TABLE Product (
#     ProductID INT PRIMARY KEY AUTO_INCREMENT,
#     ProductName VARCHAR(255) NOT NULL,
#     Description TEXT,
#     Branch VARCHAR(255),
#     Category VARCHAR(255)
# );


# CREATE TABLE Shop (
#     ShopID INT PRIMARY KEY AUTO_INCREMENT,
#     ShopName VARCHAR(255) NOT NULL,
#     Location VARCHAR(255)
# );


# CREATE TABLE PriceHistory (
#     PriceID INT PRIMARY KEY AUTO_INCREMENT,
#     ProductID INT NOT NULL,
#     ShopID INT NOT NULL,
#     Price DECIMAL(10, 2) NOT NULL,
#     EffectiveDate DATE NOT NULL,
#     FOREIGN KEY (ProductID) REFERENCES Product(ProductID),
#     FOREIGN KEY (ShopID) REFERENCES Shop(ShopID)
# );


# CREATE TABLE ProductHistory (
#     HistoryID INT PRIMARY KEY AUTO_INCREMENT,
#     ProductID INT NOT NULL,
#     EcoScore VARCHAR(10),
#     NutriScore VARCHAR(10),
#     EffectiveDate DATE NOT NULL,
#     FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
# );


In [7]:
product_description_mask = [
    'productId', 
    'name', 
    'LongName', 
    'ShortName', 
    'description', 
    'topCategoryName', 
    'topCategoryId', 
    'brand', 
    'seoBrand'
]
product_variables_mask = [
    'run_date',
    'productId', 
    'isAvailable', 
    'isPriceAvailable', 
    'inPromo', 
    'IsBio', 
    'CountryOfOrigin', 
    'IsNew', 
    'content', 
    'nutriscoreLabel', 
    'EcoscoreLabel', 
    'EcoscoreValue'
]

price_mask = [
    'run_date',
    'productId', 
    'shop_id',
    'price.basicPrice', 
    'price.recommendedQuantity', 
    'price.quantityPrice', 
    'price.quantityPriceQuantity', 
    'price.measurementUnitPrice', 
    'price.measurementUnitQuantityPrice', 
    'price.measurementUnit', 
    'price.isRedPrice', 
    'price.pricePerUOM', 
    'price.activationDate', 
    'price.recordSource', 
    'price.isPromoActive', 
    'price.priceChangeCode', 
    'price.quantityPricePerUOM', 
    'price.quantityActivationDate', 
    'price.quantityPriceChangeCode', 
    'price.redPriceReason', 
    'price.referencePrice', 
    'price.retentionPeriod', 
    'price.unit', 
    # 'promotion'
]



In [8]:

product_description_columns = {
    'productId': 'productId', 
    'name': 'name', 
    'LongName': 'longName', 
    'ShortName': 'shortName', 
    'description': 'description', 
    'topCategoryName': 'topCategoryName', 
    'topCategoryId': 'topCategoryId', 
    'brand': 'brand', 
    'seoBrand': 'seoBrand'
}

product_variables_columns = {
    'run_date': "effectiveDate",
    'productId': 'productId', 
    'isAvailable': 'isAvailable', 
    'isPriceAvailable': 'isPriceAvailable', 
    'inPromo': 'inPromo', 
    'IsBio': 'isBio', 
    'CountryOfOrigin': 'countryOfOrigin', 
    'IsNew': 'isNew', 
    'content': 'content', 
    'nutriscoreLabel': 'nutriscoreLabel', 
    'EcoscoreLabel': 'ecoscoreLabel', 
    'EcoscoreValue': 'ecoscoreValue'
}

price_columns = {
    'price_id':"priceId",
    'run_date': "effectiveDate",
    'productId': 'productId', 
    'shop_id': 'shopId',
    'price.basicPrice': 'basicPrice', 
    'price.recommendedQuantity': 'recommendedQuantity', 
    'price.quantityPrice': 'quantityPrice', 
    'price.quantityPriceQuantity': 'quantityPriceQuantity', 
    'price.measurementUnitPrice': 'measurementUnitPrice', 
    'price.measurementUnitQuantityPrice': 'measurementUnitQuantityPrice', 
    'price.measurementUnit': 'measurementUnit', 
    'price.isRedPrice': 'isRedPrice', 
    'price.pricePerUOM': 'pricePerUOM', 
    'price.activationDate': 'activationDate', 
    'price.recordSource': 'recordSource', 
    'price.isPromoActive': 'isPromoActive', 
    'price.priceChangeCode': 'priceChangeCode', 
    'price.quantityPricePerUOM': 'quantityPricePerUOM', 
    'price.quantityActivationDate': 'quantityActivationDate', 
    'price.quantityPriceChangeCode': 'quantityPriceChangeCode', 
    'price.redPriceReason': 'redPriceReason', 
    'price.referencePrice': 'referencePrice', 
    'price.retentionPeriod': 'retentionPeriod', 
    'price.unit': 'unit'
}



In [10]:
import hashlib

products_df = result_df[product_description_mask].drop_duplicates()
products_df = products_df.where(pd.notnull(products_df), None)
products_df.rename(columns=product_description_columns, inplace=True)
products_df['productId'] = products_df['productId'].astype(int)
# products_df = products_df.set_index(["productId"], drop=True)

products_history_df = result_df[product_variables_mask].drop_duplicates()
products_history_df.rename(columns=product_variables_columns, inplace=True)

prices_df = result_df[price_mask]#.drop_duplicates()
prices_df.rename(columns=price_columns, inplace=True)
prices_df['isRedPrice'] = prices_df['isRedPrice'].fillna(False)
prices_df['shopId'] = prices_df['shopId'].astype(int)
prices_df['productId'] = prices_df['productId'].astype(int)

shops_df = pd.DataFrame(result_df['shop_id'].unique(), columns=['shopId'])
# Placeholder columns for Shop details (to be updated/filled elsewhere)
shops_df['shopName'] = np.nan
shops_df['address'] = np.nan
shops_df['city'] = np.nan
# shops_df = shops_df.set_index(["shopId"], drop=True)

# Resetting indexes
products_df.reset_index(drop=True, inplace=True)
products_history_df.reset_index(drop=True, inplace=True)
prices_df.reset_index(drop=True, inplace=True)
shops_df.reset_index(drop=True, inplace=True)

prices_df['combined'] = prices_df['productId'].astype(str) + '-' + prices_df['shopId'].astype(str) + '-' + prices_df['effectiveDate'].astype(str)
prices_df['priceId'] = prices_df['combined'].apply(lambda x: hash(x)) # TODO consider using a stable hash function like hashlib or any other stable hashing mechanism.
# prices_df = prices_df.set_index('priceId')
prices_df.drop('combined', axis=1, inplace=True)

exclude_columns = ['priceId', 'effectiveDate']

# Create a hash function
def hash_row(row):
    row_str = ''.join(row.astype(str))
    return hashlib.md5(row_str.encode()).hexdigest()

# Apply the hash function to each row, excluding columns X, Y, Z
prices_df['hash_column'] = prices_df.drop(columns=exclude_columns).apply(hash_row, axis=1)



products_history_df['combined'] = products_history_df['productId'].astype(str) + '-' + products_history_df['effectiveDate'].astype(str)
products_history_df['productHistoryId'] = products_history_df['combined'].apply(lambda x: hash(x))
# products_history_df = products_history_df.set_index('productHistoryId')
products_history_df.drop('combined', axis=1, inplace=True)
exclude_columns = ['productHistoryId', 'effectiveDate']
products_history_df['hash_column'] = products_history_df.drop(columns=exclude_columns).apply(hash_row, axis=1)



/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_1641/556921551.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df.rename(columns=price_columns, inplace=True)
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_1641/556921551.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prices_df['isRedPrice'] = prices_df['isRedPrice'].fillna(False)
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_1641/556921551.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [11]:
products_history_df.head()

,effectiveDate,productId,isAvailable,isPriceAvailable,inPromo,isBio,countryOfOrigin,isNew,content,nutriscoreLabel,ecoscoreLabel,ecoscoreValue,productHistoryId,hash_column
0,2024-06-23,1038,True,True,False,False,BELGIQUE,False,NaN,NaN,NaN,NaN,-3629567933791431864,ddef41f03a77a1fb5d9134a161b02b49
1,2024-06-23,2205,True,True,False,False,COLOMBIE,False,±1kg,A,NaN,NaN,-4854241718261369370,13edf246a2ba5065351c0584662dcb7b
2,2024-06-23,2204,True,True,False,False,GUATEMALA,False,±1kg,A,NaN,NaN,-3815311478098268569,3ae5d84c4a6cae8a33177a69d05c4ec4
3,2024-06-23,2639,True,True,False,False,BELGIQUE,False,500g,A,NaN,NaN,-8159435126234019226,7258ef6942febe6954ff48bd0ee24234
4,2024-06-23,1027,True,True,False,False,ESPAGNE,False,NaN,NaN,NaN,NaN,46357078011708435,81250fd680673144af31929c144f0498


In [173]:
# # Assuming prices_df is your DataFrame
# # Convert 'effectiveDate' to datetime
# prices_df['effectiveDate'] = pd.to_datetime(prices_df['effectiveDate'], errors='coerce')

# # Convert 'productId', 'shopId', 'priceId' to int
# prices_df['productId'] = prices_df['productId'].astype(int)
# prices_df['shopId'] = prices_df['shopId'].astype(int)
# prices_df['priceId'] = prices_df['priceId'].astype(int)

# # Convert 'basicPrice', 'quantityPrice', 'measurementUnitPrice', 'measurementUnitQuantityPrice', 'pricePerUOM', 'referencePrice', 'retentionPeriod', 'quantityPricePerUOM' to float
# prices_df['basicPrice'] = prices_df['basicPrice'].astype(float)
# prices_df['quantityPrice'] = prices_df['quantityPrice'].astype(float)
# prices_df['measurementUnitPrice'] = prices_df['measurementUnitPrice'].astype(float)
# prices_df['measurementUnitQuantityPrice'] = prices_df['measurementUnitQuantityPrice'].astype(float)
# prices_df['pricePerUOM'] = prices_df['pricePerUOM'].astype(float)
# prices_df['referencePrice'] = prices_df['referencePrice'].astype(float)
# prices_df['retentionPeriod'] = prices_df['retentionPeriod'].astype(float)
# prices_df['quantityPricePerUOM'] = prices_df['quantityPricePerUOM'].astype(float)
# prices_df['recommendedQuantity'] = prices_df['recommendedQuantity'].astype(float)


# # Convert 'recommendedQuantity', 'quantityPriceQuantity', 'measurementUnit', 'activationDate', 'recordSource', 'isPromoActive', 'priceChangeCode', 'quantityActivationDate', 'quantityPriceChangeCode', 'redPriceReason', 'unit' to string (object type in pandas)
# prices_df['quantityPriceQuantity'] = prices_df['quantityPriceQuantity'].astype(str)
# prices_df['measurementUnit'] = prices_df['measurementUnit'].astype(str)
# prices_df['activationDate'] = pd.to_datetime(prices_df['activationDate'], errors='coerce')
# prices_df['recordSource'] = prices_df['recordSource'].astype(str)
# prices_df['isPromoActive'] = prices_df['isPromoActive'].astype(str)
# prices_df['priceChangeCode'] = prices_df['priceChangeCode'].astype(str)
# prices_df['quantityActivationDate'] = pd.to_datetime(prices_df['quantityActivationDate'], errors='coerce')
# prices_df['quantityPriceChangeCode'] = prices_df['quantityPriceChangeCode'].astype(str)
# prices_df['redPriceReason'] = prices_df['redPriceReason'].astype(str)
# prices_df['unit'] = prices_df['unit'].astype(str)

# # Convert 'isRedPrice' to bool
# prices_df['isRedPrice'] = prices_df['isRedPrice'].astype(bool)

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_82362/351219473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['effectiveDate'] = pd.to_datetime(prices_df['effectiveDate'], errors='coerce')
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_82362/351219473.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['productId'] = prices_df['productId'].astype(int)
/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_82362/351219473.py:7: SettingWithCopyWarning: 
A v

In [175]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55832 entries, 0 to 55831
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   effectiveDate                 55832 non-null  datetime64[ns]
 1   productId                     55832 non-null  int64         
 2   shopId                        55832 non-null  int64         
 3   basicPrice                    46308 non-null  float64       
 4   recommendedQuantity           55832 non-null  object        
 5   quantityPrice                 6441 non-null   float64       
 6   quantityPriceQuantity         55832 non-null  object        
 7   measurementUnitPrice          46300 non-null  float64       
 8   measurementUnitQuantityPrice  6441 non-null   float64       
 9   measurementUnit               55832 non-null  object        
 10  isRedPrice                    55832 non-null  bool          
 11  pricePerUOM                 

In [70]:
products_df

,productId,name,longName,shortName,description,topCategoryName,topCategoryId,brand,seoBrand
0,1038,concombre,concombre,CONCOMBRES PIECE,"*Origine: Belgique, Pays-Bas\n*Classe 1",Légumes et fruits,1675,None,None
1,2205,bananes,EVERYDAY bananes ±1kg,BANANES EVERYDAY 1KG P,"*Origine: Équateur, Colombie,\nCosta Rica\n*Cl...",Légumes et fruits,1675,EVERYDAY,Everyday
2,2204,bananes,BONI bananes ±1kg,BONI BANANEN W,"*Origine: Guatemala, Équateur,\nCosta Rica, Co...",Légumes et fruits,1675,BONI,Boni Selection
3,2639,fraises,fraises 500g,FRAISES A 500GR,"*Origine: Belgique,\nPays-Bas\n*Classe 1",Légumes et fruits,1675,None,None
4,1027,courgettes,courgettes,COURGETTES P,"*Origine: Espagne,\nBelgique, Pays-Bas\n*Classe 1",Légumes et fruits,1675,None,None
...,...,...,...,...,...,...,...,...,...
13958,158010,sauce burger TD,DEVOS LEMMENS sauce burger TD 300ml,DVL SAUCE BURGER SQ300,*Sauce burger\n*Enrichie de petits\nmorceaux d...,Epicerie,233,DEVOS LEMMENS,Devos Lemmens
13959,884128,mélange biscuits dessert frais,BONI mélange biscuits dessert frais 300g,BONI MEL.BISC.FRAIS300G,None,Boulangerie/Petit-déjeuner,124,BONI,Boni Selection
13960,10888,bouillon ext.viande boeuf bou,OXO bouillon ext.viande boeuf bou 240ml,OXO BOUILLON BOEUF240ML,*Bouillon liquide\n*240ml\n*Bouillon à boire o...,Epicerie,233,OXO,Oxo
13961,778923,provolone piquant,BRAZZALE provolone piquant 300g,BRAZ PROVO. PIQUANT300G,*Italie,Crémerie,65,BRAZZALE,Brazzale


In [155]:
prices_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55832 entries, 0 to 55831
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   effectiveDate                 55832 non-null  object 
 1   productId                     55832 non-null  int64  
 2   shopId                        55832 non-null  int64  
 3   basicPrice                    46308 non-null  float64
 4   recommendedQuantity           46308 non-null  object 
 5   quantityPrice                 6441 non-null   float64
 6   quantityPriceQuantity         6441 non-null   object 
 7   measurementUnitPrice          46300 non-null  float64
 8   measurementUnitQuantityPrice  6441 non-null   float64
 9   measurementUnit               46303 non-null  object 
 10  isRedPrice                    55832 non-null  bool   
 11  pricePerUOM                   46300 non-null  float64
 12  activationDate                46308 non-null  object 
 13  r

In [156]:
prices_df.select_dtypes(include='float').columns

Index(['basicPrice', 'quantityPrice', 'measurementUnitPrice',
       'measurementUnitQuantityPrice', 'pricePerUOM', 'quantityPricePerUOM',
       'referencePrice', 'retentionPeriod'],
      dtype='object')

In [72]:
products_df.head()
# products_df.columns

,productId,name,longName,shortName,description,topCategoryName,topCategoryId,brand,seoBrand
0,1038,concombre,concombre,CONCOMBRES PIECE,"*Origine: Belgique, Pays-Bas\n*Classe 1",Légumes et fruits,1675,None,None
1,2205,bananes,EVERYDAY bananes ±1kg,BANANES EVERYDAY 1KG P,"*Origine: Équateur, Colombie,\nCosta Rica\n*Cl...",Légumes et fruits,1675,EVERYDAY,Everyday
2,2204,bananes,BONI bananes ±1kg,BONI BANANEN W,"*Origine: Guatemala, Équateur,\nCosta Rica, Co...",Légumes et fruits,1675,BONI,Boni Selection
3,2639,fraises,fraises 500g,FRAISES A 500GR,"*Origine: Belgique,\nPays-Bas\n*Classe 1",Légumes et fruits,1675,None,None
4,1027,courgettes,courgettes,COURGETTES P,"*Origine: Espagne,\nBelgique, Pays-Bas\n*Classe 1",Légumes et fruits,1675,None,None


In [73]:
products_history_df.head()
# products_history_df.columns

,effectiveDate,productId,isAvailable,isPriceAvailable,inPromo,isBio,countryOfOrigin,isNew,content,nutriscoreLabel,ecoscoreLabel,ecoscoreValue,productHistoryId
0,2024-06-23,1038,True,True,False,False,BELGIQUE,False,NaN,NaN,NaN,NaN,1549238233022696141
1,2024-06-23,2205,True,True,False,False,COLOMBIE,False,±1kg,A,NaN,NaN,-5947574774210726444
2,2024-06-23,2204,True,True,False,False,GUATEMALA,False,±1kg,A,NaN,NaN,-8710021773845512739
3,2024-06-23,2639,True,True,False,False,BELGIQUE,False,500g,A,NaN,NaN,-6708230027703615088
4,2024-06-23,1027,True,True,False,False,ESPAGNE,False,NaN,NaN,NaN,NaN,-6475089176935072900


In [172]:
prices_df['recommendedQuantity'].unique()

array(['1.0', '1.2', '6.0', nan, '12.0', '0.6', '15.0', '8.0', '0.4',
       '0.2', '4.0', '0.5', '2.0', '3.0', '24.0', '0.9', '0.8', '9.0',
       '1.5', '0.3', '0.7', '11.0', '0.1', '5.0', '1.7', '1.3', '25.0',
       '2.2', '1.1'], dtype=object)

In [75]:
shops_df.head()
# shops_df.columns

,shopId,shopName,address,city
0,459,NaN,NaN,NaN
1,1825,NaN,NaN,NaN
2,471,NaN,NaN,NaN
3,1671,NaN,NaN,NaN


In [76]:
list(price_columns.values())

['priceId',
 'effectiveDate',
 'productId',
 'shopId',
 'basicPrice',
 'recommendedQuantity',
 'quantityPrice',
 'quantityPriceQuantity',
 'measurementUnitPrice',
 'measurementUnitQuantityPrice',
 'measurementUnit',
 'isRedPrice',
 'pricePerUOM',
 'activationDate',
 'recordSource',
 'isPromoActive',
 'priceChangeCode',
 'quantityPricePerUOM',
 'quantityActivationDate',
 'quantityPriceChangeCode',
 'redPriceReason',
 'referencePrice',
 'retentionPeriod',
 'unit']

In [77]:

# Establish database connection
db = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

cursor = db.cursor()


In [78]:

# Retrieve current products and shops from the database
cursor.execute("SELECT productId FROM Product")
existing_products = {row[0] for row in cursor.fetchall()}

cursor.execute("SELECT shopId FROM Shop")
existing_shops = {row[0] for row in cursor.fetchall()}

# Insert new products if they do not exist
new_products = products_df[~products_df['productId'].astype(int).isin(existing_products)]
for _, row in new_products.iterrows():
    cursor.execute("""
        INSERT INTO Product (productId, name, longName, shortName, description, topCategoryName, topCategoryId, brand, seoBrand)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['productId'], row['name'], row['longName'], row['shortName'], row['description'], row['topCategoryName'], 
          row['topCategoryId'], row['brand'], row['seoBrand']))

# Insert new shops if they do not exist
new_shops = shops_df[~shops_df['shopId'].astype(int).isin(existing_shops)]
for shop_id in new_shops['shopId'].unique():
    # Example shop data, adjust as per actual data
    shop_name = 'Unknown name'
    address = 'Unknown address'
    city = 'Unknown city'
    cursor.execute("""
        INSERT INTO Shop (shopId, shopName, address, city)
        VALUES (%s, %s, %s, %s)
    """, (shop_id, shop_name, address, city))

db.commit()

cursor.close()
db.close()

In [169]:

db = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

cursor = db.cursor()
# Retrieve current prices from the database
cursor.execute("""
    SELECT PriceId, EffectiveDate, ProductID, ShopID, BasicPrice, RecommendedQuantity, QuantityPrice, QuantityPriceQuantity,
           MeasurementUnitPrice, MeasurementUnitQuantityPrice, MeasurementUnit, IsRedPrice, PricePerUOM,
           ActivationDate, RecordSource, IsPromoActive, PriceChangeCode, QuantityPricePerUOM,
           QuantityActivationDate, QuantityPriceChangeCode, RedPriceReason, ReferencePrice, RetentionPeriod,
           Unit 
    FROM PriceHistory
    WHERE (ProductID, ShopID, EffectiveDate) IN (
        SELECT ProductID, ShopID, MAX(EffectiveDate)
        FROM PriceHistory
        GROUP BY ProductID, ShopID
    )
""")
current_prices = cursor.fetchall()
current_prices_df = pd.DataFrame(current_prices, columns= list(price_columns.values())).drop("effectiveDate", axis=1)


# Round floating-point numbers to a consistent precision
float_cols_current = current_prices_df.select_dtypes(include='float').columns
float_cols_prices = prices_df.select_dtypes(include='float').columns
current_prices_df[float_cols_current] = current_prices_df[float_cols_current].round(2)
prices_df[float_cols_prices] = prices_df[float_cols_prices].round(2)

# current_prices_df.replace({np.nan: None, 'nan': None, 'NaN': None}, inplace=True)
# prices_df.replace({np.nan: None, 'nan': None, 'NaN': None}, inplace=True)

merged_prices = prices_df.merge(current_prices_df, on=['productId', 'shopId'], how='left', suffixes=('_new', '_old'))


# Retrieve current product modifications from the database
cursor.execute("""
    SELECT EffectiveDate, ProductID, IsAvailable, IsPriceAvailable, InPromo, IsBio, CountryOfOrigin, IsNew, Content,
           NutriscoreLabel, EcoscoreLabel, EcoscoreValue
    FROM ProductHistory
    WHERE (ProductID, EffectiveDate) IN (
        SELECT ProductID, MAX(EffectiveDate)
        FROM ProductHistory
        GROUP BY ProductID
    )
""")

current_product_modifications = cursor.fetchall()
current_product_modifications_df = pd.DataFrame(current_product_modifications, columns=list(product_variables_columns.values())).drop("effectiveDate", axis=1)

merged_products = products_history_df.merge(current_product_modifications_df, on='productId', how='left', suffixes=('_new', '_old'))


# Function to create a hashable string from DataFrame rows
def create_hashable_string(row):
    return ''.join(row.astype(str))

cursor.close()
db.close()

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_82362/223878313.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df[float_cols_prices] = prices_df[float_cols_prices].round(2)


In [170]:
float_cols_prices

Index(['basicPrice', 'quantityPrice', 'measurementUnitPrice',
       'measurementUnitQuantityPrice', 'pricePerUOM', 'quantityPricePerUOM',
       'referencePrice', 'retentionPeriod'],
      dtype='object')

In [171]:
merged_prices.head()


,effectiveDate,productId,shopId,basicPrice_new,recommendedQuantity_new,quantityPrice_new,quantityPriceQuantity_new,measurementUnitPrice_new,measurementUnitQuantityPrice_new,measurementUnit_new,isRedPrice_new,pricePerUOM_new,activationDate_new,recordSource_new,isPromoActive_new,priceChangeCode_new,quantityPricePerUOM_new,quantityActivationDate_new,quantityPriceChangeCode_new,redPriceReason_new,referencePrice_new,retentionPeriod_new,unit_new,priceId_new,priceId_old,basicPrice_old,recommendedQuantity_old,quantityPrice_old,quantityPriceQuantity_old,measurementUnitPrice_old,measurementUnitQuantityPrice_old,measurementUnit_old,isRedPrice_old,pricePerUOM_old,activationDate_old,recordSource_old,isPromoActive_old,priceChangeCode_old,quantityPricePerUOM_old,quantityActivationDate_old,quantityPriceChangeCode_old,redPriceReason_old,referencePrice_old,retentionPeriod_old,unit_old
0,2024-06-23,1038,459,0.79,1.0,0.6,2.0,0.79,0.6,S,False,0.79,18-06-2024,Offline,N,L,0.6,18-06-2024,L,NaN,NaN,NaN,NaN,5527235774738387675,-642738901272015156,0.79,1.00,0.60,2.00,0.79,0.60,S,False,0.79,18-06-2024,Offline,N,L,0.60,18-06-2024,L,NaN,NaN,NaN,NaN
1,2024-06-23,1038,459,0.79,1.0,0.6,2.0,0.79,0.6,S,False,0.79,18-06-2024,Offline,N,L,0.6,18-06-2024,L,NaN,NaN,NaN,NaN,5527235774738387675,5527235774738387675,0.79,1.00,0.60,2.00,0.79,0.60,S,False,0.79,18-06-2024,Offline,N,L,0.60,18-06-2024,L,NaN,NaN,NaN,NaN
2,2024-06-23,2205,459,1.29,1.0,NaN,NaN,1.29,NaN,K,False,1.29,15-05-2024,Offline,N,L,NaN,NaN,NaN,NaN,NaN,NaN,K,1949494219402720820,-2325100244627051593,1.29,1.00,NaN,NaN,1.29,NaN,K,False,1.29,15-05-2024,Offline,N,L,NaN,NaN,NaN,NaN,NaN,NaN,K
3,2024-06-23,2205,459,1.29,1.0,NaN,NaN,1.29,NaN,K,False,1.29,15-05-2024,Offline,N,L,NaN,NaN,NaN,NaN,NaN,NaN,K,1949494219402720820,1949494219402720820,1.29,1.00,NaN,NaN,1.29,NaN,K,False,1.29,15-05-2024,Offline,N,L,NaN,NaN,NaN,NaN,NaN,NaN,K
4,2024-06-23,2204,459,1.99,1.2,NaN,NaN,1.99,NaN,K,False,1.99,20-06-2024,Offline,N,E,NaN,NaN,NaN,NaN,NaN,NaN,K,-8635023651859170240,-3838020152584693141,1.99,1.20,NaN,NaN,1.99,NaN,K,False,1.99,20-06-2024,Offline,N,E,NaN,NaN,NaN,NaN,NaN,NaN,K


In [131]:
current_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111664 entries, 0 to 111663
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   priceId                       111664 non-null  object
 1   productId                     111664 non-null  int64 
 2   shopId                        111664 non-null  int64 
 3   basicPrice                    92616 non-null   object
 4   recommendedQuantity           92616 non-null   object
 5   quantityPrice                 12882 non-null   object
 6   quantityPriceQuantity         12882 non-null   object
 7   measurementUnitPrice          92600 non-null   object
 8   measurementUnitQuantityPrice  12882 non-null   object
 9   measurementUnit               111664 non-null  object
 10  isRedPrice                    111664 non-null  bool  
 11  pricePerUOM                   92600 non-null   object
 12  activationDate                111664 non-null  object
 13 

In [150]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55832 entries, 0 to 55831
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   effectiveDate                 55832 non-null  object
 1   productId                     55832 non-null  int64 
 2   shopId                        55832 non-null  int64 
 3   basicPrice                    46308 non-null  object
 4   recommendedQuantity           46308 non-null  object
 5   quantityPrice                 6441 non-null   object
 6   quantityPriceQuantity         6441 non-null   object
 7   measurementUnitPrice          46300 non-null  object
 8   measurementUnitQuantityPrice  6441 non-null   object
 9   measurementUnit               46303 non-null  object
 10  isRedPrice                    55832 non-null  bool  
 11  pricePerUOM                   46300 non-null  object
 12  activationDate                46308 non-null  object
 13  recordSource    

In [132]:
merged_prices.columns

Index(['effectiveDate', 'productId', 'shopId', 'basicPrice_new',
       'recommendedQuantity_new', 'quantityPrice_new',
       'quantityPriceQuantity_new', 'measurementUnitPrice_new',
       'measurementUnitQuantityPrice_new', 'measurementUnit_new',
       'isRedPrice_new', 'pricePerUOM_new', 'activationDate_new',
       'recordSource_new', 'isPromoActive_new', 'priceChangeCode_new',
       'quantityPricePerUOM_new', 'quantityActivationDate_new',
       'quantityPriceChangeCode_new', 'redPriceReason_new',
       'referencePrice_new', 'retentionPeriod_new', 'unit_new', 'priceId_new',
       'priceId_old', 'basicPrice_old', 'recommendedQuantity_old',
       'quantityPrice_old', 'quantityPriceQuantity_old',
       'measurementUnitPrice_old', 'measurementUnitQuantityPrice_old',
       'measurementUnit_old', 'isRedPrice_old', 'pricePerUOM_old',
       'activationDate_old', 'recordSource_old', 'isPromoActive_old',
       'priceChangeCode_old', 'quantityPricePerUOM_old',
       'quantityActiv

In [134]:
# Add a comparison column to check for changes in price data
merged_prices['comparison_new'] = merged_prices.apply(lambda row: create_hashable_string(row.filter(like='_new').drop(labels=row.filter(like='priceId_new').index)), axis=1)
merged_prices['comparison_old'] = merged_prices.apply(lambda row: create_hashable_string(row.filter(like='_old').drop(labels=row.filter(like='priceId_old').index)), axis=1)

# Identify changed rows in price data
price_changes = merged_prices[merged_prices['comparison_new'] != merged_prices['comparison_old']]


In [120]:

# Add a comparison column to check for changes in product data
merged_products['comparison_new'] = merged_products.apply(lambda row: create_hashable_string(row.filter(regex= '(_new|priceId)')), axis=1)
merged_products['comparison_old'] = merged_products.apply(lambda row: create_hashable_string(row.filter(regex= '(_old|priceId)')), axis=1)

# Identify changed rows in product data
product_changes = merged_products[merged_products['comparison_new'] != merged_products['comparison_old']]

In [128]:
price_changes.head(1)

,effectiveDate,productId,shopId,basicPrice_new,recommendedQuantity_new,quantityPrice_new,quantityPriceQuantity_new,measurementUnitPrice_new,measurementUnitQuantityPrice_new,measurementUnit_new,isRedPrice_new,pricePerUOM_new,activationDate_new,recordSource_new,isPromoActive_new,priceChangeCode_new,quantityPricePerUOM_new,quantityActivationDate_new,quantityPriceChangeCode_new,redPriceReason_new,referencePrice_new,retentionPeriod_new,unit_new,priceId_new,priceId_old,basicPrice_old,recommendedQuantity_old,quantityPrice_old,quantityPriceQuantity_old,measurementUnitPrice_old,measurementUnitQuantityPrice_old,measurementUnit_old,isRedPrice_old,pricePerUOM_old,activationDate_old,recordSource_old,isPromoActive_old,priceChangeCode_old,quantityPricePerUOM_old,quantityActivationDate_old,quantityPriceChangeCode_old,redPriceReason_old,referencePrice_old,retentionPeriod_old,unit_old,comparison_new,comparison_old
0,2024-06-23,1038,459,0.79,1.0,0.595,2.0,0.79,0.6,S,False,0.79,18-06-2024,Offline,N,L,0.6,18-06-2024,L,NaN,NaN,NaN,NaN,5527235774738387675,-642738901272015156,0.79,1.00,0.60,2.00,0.79,0.60,S,False,0.79,18-06-2024,Offline,N,L,0.60,18-06-2024,L,NaN,NaN,NaN,NaN,0.791.00.5952.00.790.6SFalse0.7918-06-2024Offl...,0.791.000.602.000.790.60SFalse0.7918-06-2024Of...


In [135]:
price_changes['comparison_new'].iloc[0]

'0.791.00.5952.00.790.6SFalse0.7918-06-2024OfflineNL0.618-06-2024LNoneNoneNoneNone'

In [136]:
price_changes['comparison_old'].iloc[0]

'0.791.000.602.000.790.60SFalse0.7918-06-2024OfflineNL0.6018-06-2024LNoneNoneNoneNone'

In [ ]:
# Insert new prices if there is a change

for _, row in price_changes.iterrows():
    print(row)
    # Ensure all values are of correct types
    values = (
        row['effectiveDate_new'], row['productId'], row['shopId'], row['basicPrice_new'], row['recommendedQuantity_new'],
        row['quantityPrice_new'], row['quantityPriceQuantity_new'], row['measurementUnitPrice_new'],
        row['measurementUnitQuantityPrice_new'], row['measurementUnit_new'], row['isRedPrice_new'],
        row['pricePerUOM_new'], row['activationDate_new'], row['recordSource_new'],
        row['isPromoActive_new'], row['priceChangeCode_new'], row['quantityPricePerUOM_new'],
        row['quantityActivationDate_new'], row['quantityPriceChangeCode_new'], row['redPriceReason_new'],
        row['referencePrice_new'], row['retentionPeriod_new'], row['unit_new'], row['priceId'],
        # row['promotion_new'],
    )
    
    cursor.execute("""
        INSERT INTO PriceHistory (EffectiveDate, ProductID, ShopID, BasicPrice, RecommendedQuantity, QuantityPrice,
                                  QuantityPriceQuantity, MeasurementUnitPrice, MeasurementUnitQuantityPrice,
                                  MeasurementUnit, IsRedPrice, PricePerUOM, ActivationDate, RecordSource,
                                  IsPromoActive, PriceChangeCode, QuantityPricePerUOM, QuantityActivationDate,
                                  QuantityPriceChangeCode, RedPriceReason, ReferencePrice, RetentionPeriod,
                                  Unit, PriceId)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, values)

# Insert new product modifications if there is a change
for _, row in product_changes.iterrows():
    cursor.execute("""
        INSERT INTO ProductHistory (EffectiveDate, ProductID, IsAvailable, IsPriceAvailable, InPromo, IsBio,
                                    CountryOfOrigin, IsNew, Content, NutriscoreLabel, EcoscoreLabel, EcoscoreValue)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (datetime.now().date(), row['productId'], row['isAvailable_new'], row['isPriceAvailable_new'], row['inPromo_new'],
          row['isBio_new'], row['countryOfOrigin_new'], row['isNew_new'], row['content_new'], row['nutriscoreLabel_new'],
          row['ecoscoreLabel_new'], row['ecoscoreValue_new']))

# Commit the transaction
db.commit()


In [39]:

# Close the cursor and connection
cursor.close()
db.close()


In [28]:
new_shops

,shopId,shopName,address,city
0,459,NaN,NaN,NaN
1,1825,NaN,NaN,NaN
2,471,NaN,NaN,NaN
3,1671,NaN,NaN,NaN


In [4]:
price_changes

NameError: name 'price_changes' is not defined

In [100]:
products_history_df.head()


,effectiveDate,productId,isAvailable,isPriceAvailable,inPromo,isBio,countryOfOrigin,isNew,content,nutriscoreLabel,ecoscoreLabel,ecoscoreValue,productHistoryId,comparison
0,2024-06-23,1038,True,True,False,False,BELGIQUE,False,NaN,NaN,NaN,NaN,-308741590286644864,2024-06-231038TrueTrueFalseFalseBELGIQUEFalsen...
1,2024-06-23,2205,True,True,False,False,COLOMBIE,False,±1kg,A,NaN,NaN,-1856504227285816053,2024-06-232205TrueTrueFalseFalseCOLOMBIEFalse±...
2,2024-06-23,2204,True,True,False,False,GUATEMALA,False,±1kg,A,NaN,NaN,-2026812429116595126,2024-06-232204TrueTrueFalseFalseGUATEMALAFalse...
3,2024-06-23,2639,True,True,False,False,BELGIQUE,False,500g,A,NaN,NaN,-3664904188612572528,2024-06-232639TrueTrueFalseFalseBELGIQUEFalse5...
4,2024-06-23,1027,True,True,False,False,ESPAGNE,False,NaN,NaN,NaN,NaN,-7292339018932774172,2024-06-231027TrueTrueFalseFalseESPAGNEFalsena...


In [101]:
current_product_modifications_df.head()

,productId,name,longName,shortName,description,topCategoryName,topCategoryId,brand,seoBrand,comparison
